# Runs through the nuScenes dataset

In [ ]:
from avapi.nuscenes import nuScenesManager


# initialize scene manager
nusc_data_dir = "../../data/nuScenes"
NSM = nuScenesManager(nusc_data_dir, split="v1.0-mini")  # v1.0-trainval for full

## Loops over nuscenes, applies perception and tracking

In [ ]:
from avstack.modules.assignment import build_A_from_iou, gnn_single_frame_assign


def compute_assignment_metrics(observed, truth):
    """Compute the metrics between a set of objects and truths"""

    # perform assignment
    A = build_A_from_iou(observed, truth)
    assign = gnn_single_frame_assign(A, algorithm="JVC", cost_threshold=-0.10)
    iou_assign = [-A[a[0], a[1]] for a in assign.assignment_tuples]

    # package up the metrics
    metrics = {
        "n_observed": len(observed),
        "n_truth": len(truth),
        "n_assignment": len(assign.assignment_tuples),
        "iou_of_assign": iou_assign,
        "n_fp": len(observed) - len(assign.assignment_tuples),
        "n_fn": len(truth) - len(assign.assignment_tuples),
    }

    return metrics

In [ ]:
from avstack.modules.perception.object2dfv import MMDetObjectDetector2D
from avstack.modules.perception.object3d import MMDetObjectDetector3D

# set up the perception here (always the same)
# -- 2d detection from images
print("Loading image perception model")
detector_2d = MMDetObjectDetector2D(
    model="fasterrcnn",
    dataset="nuscenes",
    threshold=0.5,
    gpu=0,
    epoch="latest",
)
# -- 3d detection from lidar
print("Loading lidar perception model")
detector_3d = MMDetObjectDetector3D(
    model="pointpillars",
    dataset="nuscenes",
    threshold=0.5,
    gpu=0,
    epoch="latest",
)

In [ ]:
from avapi.visualize.snapshot import show_image_with_boxes, show_lidar_bev_with_boxes


# test out 2d and 3d detectors
NSD = NSM.get_scene_dataset_by_index(0)
frame_idx = 10
camera = "CAM_FRONT"
lidar = "LIDAR_TOP"
frame = NSD.get_frames(sensor=camera)[frame_idx]
img = NSD.get_image(frame=frame, sensor=camera)
pc = NSD.get_lidar(frame=frame, sensor=lidar)

# run inference
dets_2d = detector_2d(img)
dets_3d = detector_3d(pc)

# visualize results
show_image_with_boxes(img, boxes=dets_2d, inline=True, show=True)
show_lidar_bev_with_boxes(pc, boxes=dets_3d, inline=True, show=True)


In [ ]:
from tqdm import tqdm
from avstack.geometry import GlobalOrigin3D
from avstack.modules.tracking.tracker2d import BasicBoxTracker2D
from avstack.modules.tracking.tracker3d import BasicBoxTracker3D



# loop over scenes
metrics_scenes = {}
for scene in NSM.scenes:
    # get this scene (dataset)
    print(f"Running over scene: {scene}")
    NSD = NSM.get_scene_dataset_by_name(scene)

    # initialize new trackers
    # -- 2d box tracker
    tracker_2d = BasicBoxTracker2D(
        threshold_confirmed=2,
        threshold_coast=5,
        v_max=200,
        cost_threshold=-0.1,
        check_reference=False,  # don't check ref for 2D tracks
    )
    # -- 3d box tracker
    tracker_3d = BasicBoxTracker3D(
        threshold_confirmed=3,
        threshold_coast=5,
        v_max=60,
        assign_metric="center_dist",
        assign_radius=6,
        check_reference=True,
    )
    
    # run through the frames
    all_sensors = NSD.sensor_IDs
    camera = "CAM_FRONT"  # could also use another sensor from "all_sensors"
    lidar = "LIDAR_TOP"
    metrics_frames = {}
    for frame in tqdm(NSD.get_frames(sensor=camera)):
        # get sensor data
        ts = NSD.get_timestamp(frame=frame, sensor=camera)
        img = NSD.get_image(frame=frame, sensor=camera)
        pc = NSD.get_lidar(frame=frame, sensor=lidar)

        # run perception in the LOCAL coordinates
        dets_2d = detector_2d(img)
        dets_3d = detector_3d(pc)

        # run tracking in the GLOBAL coordinates
        tracks_2d = tracker_2d(dets_2d, platform=GlobalOrigin3D)
        tracks_3d = tracker_3d(dets_3d, platform=GlobalOrigin3D)

        # get truth locations for metrics
        objs_cam = NSD.get_objects(frame=frame, sensor=camera, max_dist=60)
        objs_lid = NSD.get_objects(frame=frame, sensor=lidar, max_dist=60)

        # compute metrics for each frame
        metrics_2d_det = compute_assignment_metrics(dets_2d, objs_cam)
        metrics_3d_det = compute_assignment_metrics(dets_3d, objs_lid)
        metrics_2d_trk = compute_assignment_metrics(tracks_2d, objs_cam)
        metrics_3d_trk = compute_assignment_metrics(tracks_3d, objs_lid)

        # store metrics
        metrics_frames[frame] = {
            "scene": scene,
            "frame": frame,
            "timestamp": ts,
            "perception": [
                {
                    "sensor": camera,
                    "metrics": metrics_2d_det,
                },
                {
                    "sensor": lidar,
                    "metrics": metrics_3d_det,
                },
            ],
            "tracking": [
                {
                    "sensor": camera,
                    "metrics": metrics_2d_trk,
                },
                {
                    "sensor": lidar,
                    "metrics": metrics_3d_trk,
                }
            ]
        }

        # NOTE: to continue to run through all the scenarios, don't include this break
        if frame > 10:
            break

    # visualize the last set of data
    show_image_with_boxes(img, boxes=dets_2d, inline=True, show=True)
    show_image_with_boxes(img, boxes=tracks_2d, inline=True, show=True)
    show_lidar_bev_with_boxes(pc, boxes=dets_3d, inline=True, show=True)
    show_lidar_bev_with_boxes(pc, boxes=tracks_3d, inline=True, show=True)

    # store the metrics for this scene
    metrics_scenes[scene] = metrics_frames

    # NOTE: to continue to run through all the scenarios, don't include this break
    break